In [66]:
from dotenv import load_dotenv
import pandas as pd
import os
import base64
from requests import post, get
import json
import tekore
import numpy as np

In [67]:
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [77]:
# Gets the access token
def get_token():
    auth_str = client_id + ":" + client_secret
    auth_bytes = auth_str.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded", 
    }

    data = {"grant_type": "client_credentials"}

    result = post(url, headers=headers, data=data)

    json_result = json.loads(result.content)
    token = json_result["access_token"]

    return token

# Gets the header that will be used in requests
def get_auth_headers(token):
    return {"Authorization": "Bearer " + token}

# Search for artists
def search_for_artists(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_headers(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query

    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)
    print(json_result)


# Gets the tracks from a playlist
def get_playlists(token, playlist_id):

    url = "https://api.spotify.com/v1/playlists"
    query_url = url + "/" + playlist_id + "/tracks"
    headers = get_auth_headers(token)

    song_dict = {}
    
    count = 0
    while True:

        result = get(query_url, headers=headers)
        json_result = json.loads(result.content)

        for item in json_result['items']:
            song_dict[count] = item
            count+=1

        query_url = json_result['next']

        if not query_url:
            break

    # Creating a DF with the tracks in then playlist
    df = pd.DataFrame.from_dict(song_dict, orient='index')
    
    print(len(df))
    
    track_dict = df['track'].values

    print(len(track_dict))

    track_information = []

    count = 0
    for i in track_dict:
        track_information.append([i['preview_url'], i['external_urls'], i['href'], i['name']])

    print(track_information)
    df['preview_url'] = [array[0] for array in track_information]
    df['external_urls'] = [array[1] for array in track_information] 
    df['href'] = [array[2] for array in track_information] 
    df['name'] = [array[3] for array in track_information] 

    df = df.drop(['track', 'primary_color', 'video_thumbnail', 'is_local'], axis=1)

    df.to_csv('spotify_data.csv')

In [78]:
token = get_token()

# search_for_artists(token, "drake")
get_playlists(token, "11gfggVyVq569PqxQHfvHq")

334
334
[[None, {'spotify': 'https://open.spotify.com/track/1UGD3lW3tDmgZfAVDh6w7r'}, 'https://api.spotify.com/v1/tracks/1UGD3lW3tDmgZfAVDh6w7r', 'Devil In A New Dress'], [None, {'spotify': 'https://open.spotify.com/track/3DK6m7It6Pw857FcQftMds'}, 'https://api.spotify.com/v1/tracks/3DK6m7It6Pw857FcQftMds', 'Runaway'], [None, {'spotify': 'https://open.spotify.com/track/22L7bfCiAkJo5xGSQgmiIO'}, 'https://api.spotify.com/v1/tracks/22L7bfCiAkJo5xGSQgmiIO', 'All Of The Lights'], [None, {'spotify': 'https://open.spotify.com/track/0IW0qaeyxL5Et4UG2MrJKB'}, 'https://api.spotify.com/v1/tracks/0IW0qaeyxL5Et4UG2MrJKB', "We Don't Care"], [None, {'spotify': 'https://open.spotify.com/track/5SkRLpaGtvYPhw02vZhQQ9'}, 'https://api.spotify.com/v1/tracks/5SkRLpaGtvYPhw02vZhQQ9', 'All Falls Down'], [None, {'spotify': 'https://open.spotify.com/track/1ko2NuvWlQdxtNRc8QQzmT'}, 'https://api.spotify.com/v1/tracks/1ko2NuvWlQdxtNRc8QQzmT', 'Spaceship'], [None, {'spotify': 'https://open.spotify.com/track/3A4cpTBP